![](resources/title.png)

# Tarea en Python usando PySpark para la asignatura de PAB

### El objetivo de la práctica es analizar los datos de indicadores de enfermedades crónicas que se proporcionan en el portal dado, de dónde obtenemos un fichero .csv que contiene los datos a analizar. 

### Usaremos la API de RDDs y la de datasets

### Autores:
* Miguel González 
* Samuel Delgado
* Victoria Cruz

## PyData.py con API RDDs

In [34]:
from pyspark import SparkConf, SparkContext
import sys
import csv
import time


file_name = "data/data.csv"

sc = SparkContext.getOrCreate()

logger = sc._jvm.org.apache.log4j
logger.LogManager.getLogger("org").setLevel(logger.Level.WARN)

start_computing_time = time.time()

file = sc \
    .textFile(file_name)
header = file.first()

formatted = file.filter(lambda line: line != header) \
    .map(lambda line: csv.reader([line], quotechar='"', delimiter=',',
                                 quoting=csv.QUOTE_ALL, skipinitialspace=True).__next__())

d = formatted.map(lambda r: (r[5], r[6])) \
    .groupByKey() \
    .map(lambda x: (x[0], set(x[1]))) \
    .mapValues(len) \
    .collect()
    
total_computing_time = time.time() - start_computing_time

for pair in d:
    print(pair)
    
print("Computing time: ", str(total_computing_time))

('Asthma', 9)
('Chronic Kidney Disease', 4)
('Older Adults', 4)
('Mental Health', 3)
('Immunization', 1)
('Disability', 1)
('Cancer', 20)
('Oral Health', 9)
('Tobacco', 16)
('Reproductive Health', 3)
('Arthritis', 10)
('Diabetes', 20)
('Cardiovascular Disease', 18)
('Overarching Conditions', 16)
('Alcohol', 16)
('Chronic Obstructive Pulmonary Disease', 16)
('Nutrition, Physical Activity, and Weight Status', 37)
Computing time:  7.092781066894531


## PyDataFrame.py con API datasets

In [33]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as func

import sys
import time

file_name = "data/data.csv"

spark_session = SparkSession \
    .builder \
    .getOrCreate()

logger = spark_session._jvm.org.apache.log4j
logger.LogManager.getLogger("org").setLevel(logger.Level.WARN)

start_computing_time = time.time()

data_frame = spark_session \
    .read \
    .format("csv") \
    .options(header='true', inferschema='true') \
    .load(file_name)

indicator_freqs = data_frame \
    .groupBy("Topic") \
    .agg(func.countDistinct("Question").alias("freq")) \
    .sort("Topic") \
    
indicator_freqs.show()

data_frame \
    .groupBy("Topic", "LocationDesc") \
    .count() \
    .sort("LocationDesc") \
    .show()

total_computing_time = time.time() - start_computing_time
print("Computing time: ", str(total_computing_time))

+--------------------+----+
|               Topic|freq|
+--------------------+----+
|             Alcohol|  16|
|           Arthritis|  10|
|              Asthma|   9|
|              Cancer|  20|
|Cardiovascular Di...|  18|
|Chronic Kidney Di...|   4|
|Chronic Obstructi...|  16|
|            Diabetes|  20|
|          Disability|   1|
|        Immunization|   1|
|       Mental Health|   3|
|Nutrition, Physic...|  37|
|        Older Adults|   4|
|         Oral Health|   9|
|Overarching Condi...|  16|
| Reproductive Health|   3|
|             Tobacco|  16|
+--------------------+----+

+--------------------+------------+-----+
|               Topic|LocationDesc|count|
+--------------------+------------+-----+
|           Arthritis|     Alabama|  768|
|Overarching Condi...|     Alabama|  733|
|             Alcohol|     Alabama|  593|
|         Oral Health|     Alabama|  205|
|Cardiovascular Di...|     Alabama| 1399|
|        Immunization|     Alabama|   96|
|Nutrition, Physic...|     Alabam

### Plot indicator frequencies.

In [ ]:
indicator_names = [x.Topic for x in indicator_freqs.select("Topic").collect()]
indicator_count = [x.freq for x in indicator_freqs.select("freq").collect()]
from matplotlib import pyplot
pyplot.figure(figsize=(12, 12))
pyplot.xticks(rotation=90)
pyplot.bar(indicator_names, indicator_count ,align='center')